## Example Points Observation Data
The simulation time of the .m21fm file in this example has been intentionally kept short. This adjustment allows for faster calculations within the notebook, making it easier to demonstrate the general usage of the package.

Please note that due to the shortened simulation period, the quality of the results is not representative of a full-scale analysis. This example is intended solely for demonstration purposes and should not be used for detailed assessments or decision-making.

##### Import

In [1]:
from pathlib import Path
from datetime import datetime

import optuna
from mike_autocal.autocal import AutoCal
from mike_autocal.dataio import ObservationData, SimObsPair, SimulationData
from mike_autocal.measurement_fun import ManningFile
from mike_autocal.mikesimulation import Launcher, RunTimeEvaluation
from mike_autocal.objective_fun import AMEANOuterMetric, RMSEInnerMetric

ROOT_DIR = Path().resolve()

##### Basic autocal input

- `study_name` - Unique identifier for this calibration study using satellite altimetry data.
- `simfile` - Base MIKE model file to be calibrated.
- `evaluation_time` - Defines from which time range is used to evaluate the output files. Can e.g. be usefull to ignore warm up.
- `n_trials` - Number of optimization iterations to perform.
- `sampler` - The optuna sampler that is use for the study.
- `direction` - Indicates that the optimization should "minimize" or "maximize" the error metrics. Can be a list if multiple objectives are used.
- `inner_metric` -  Applied to each simulation-observation pair independently. Can be a list if multiple objectives are used.
- `outer_metric` - Combines results from multiple simulation-observation pairs into a single objective value. Can be a list if multiple objectives are used.

In [2]:
study_name = "example_points"
simfile = ROOT_DIR / "mike_autocal/tests/data/simulation_data_sns/simulation/sns_base.m21fm"
evaluation_time = slice(50, None)
n_trials = 11
sampler = optuna.samplers.GPSampler(seed=0)
direction = ["minimize"]
inner_metric = [RMSEInnerMetric()]
outer_metric = [AMEANOuterMetric()]
use_gpu=True

##### Create measurement functions

Measurement functions define the parameters that will be calibrated in the MIKE model. Meaningful ranges can be defined it wished.

In [3]:
measurement_functions = [
    ManningFile(
        filename= ROOT_DIR / "mike_autocal/tests/data/simulation_data_sns/simulation/conditions/ManningM.dfsu",
        item_name="manning",
        low=0.001,
        high=81.101,
        step=0.01,
    )]

##### Create a Mapping of Simulation and Observation Data (Simulation-Observation Pairs)

In order for the autocal system to recognize the observation data as measurement points, we need to specify `pair_type="point"` in the `SimObsPair`. This indicates that the data corresponds to a point observation.

Additionally, to properly interpret the `dfs0` files associated with point-type observations, we need to specify the `item` number of the respective measurement in the file.

In [4]:
base_sim_path = ROOT_DIR / "mike_autocal/tests/data/simulation_data_sns/simulation/sns_base.m21fm - Result Files"
base_obs_path = ROOT_DIR / "mike_autocal/tests/data/simulation_data_sns/observations"

simobs = [
    SimObsPair(
        name="F3platform",
        pair_type="point",
        sim=SimulationData(
            file_path = base_sim_path / "waterlevels.dfs0",
            item=7,  
        ),
        obs=ObservationData(
            file_path = base_obs_path / "F3platform_wl.dfs0",
            item=0,  
        ),
    ),
    SimObsPair(
        name="Helgoland",
        pair_type="point",
        sim=SimulationData(
            file_path = base_sim_path / "waterlevels.dfs0",
            item=5,  
        ),
        obs=ObservationData(
            file_path = base_obs_path / "Helgoland_wl.dfs0",
            item=0,  
        ),
    ),
    SimObsPair("J61",
        pair_type="point",
        sim=SimulationData(
            file_path = base_sim_path / "waterlevels.dfs0",
            item=6,  
        ),
        obs=ObservationData(
            file_path = base_obs_path / "J61_wl.dfs0",
            item=0,  
        ),
    ),
    SimObsPair(
        name="K141",
        pair_type="point",
        sim=SimulationData(
            file_path = base_sim_path / "waterlevels.dfs0",
            item=3,  
        ),
        obs=ObservationData(
            file_path = base_obs_path / "K141_wl.dfs0",
            item=0,  
        ),
    ),
    SimObsPair(
        name="Westkapelle",
        pair_type="point",
        sim=SimulationData(
            file_path = base_sim_path / "waterlevels.dfs0",
            item=1,  
        ),
        obs=ObservationData(
            file_path = base_obs_path / "Westkapelle_wl.dfs0",
            item=0,  
        ),
    ),
    SimObsPair(
        name="ZeebruggeWielingendok_wl",
        pair_type="point",
        sim=SimulationData(
            file_path = base_sim_path / "waterlevels.dfs0",
            item=0,  
        ),
        obs=ObservationData(
            file_path = base_obs_path / "ZeebruggeWielingendok_wl.dfs0",
            item=0,  
        ),
    ),
    SimObsPair(
        name="L91",
        pair_type="point",
        sim=SimulationData(
            file_path = base_sim_path / "waterlevels.dfs0",
            item=2,  
        ),
        obs=ObservationData(
            file_path = base_obs_path / "L91_wl.dfs0",
            item=0,  
        ),
    ),]

##### Create Launcher

In [5]:
rte = RunTimeEvaluation(
    simobs=simobs,
    inner_metric=inner_metric,
    outer_metric=outer_metric,
    frequency=50,
    logdir=Path(f"logs/simulation_{datetime.now().strftime('%Y%m%d_%H%M%S')}"),
)

launcher = Launcher(
    simfile=simfile,
    use_gpu=use_gpu,
    runtimeevaluation=rte,
)

##### Create autocal object

In [6]:
calibration = AutoCal(
    study_name=study_name,
    launcher=launcher,
    simobs=simobs,
    inner_metric=inner_metric,  
    outer_metric=outer_metric,
    n_trials=n_trials,
    direction=direction,
    sampler=sampler, 
    measurement_functions=measurement_functions,
    evaluation_time=evaluation_time,
    verbose=False,
    load_if_exists=False,
)
calibration.run()

[I 2025-05-13 11:41:45,447] A new study created in Journal with name: example_points
INFO:autocal:------------------------------------ Trial: 0 ------------------------------------
Processing: 100%|██████████| 500/500 [00:13<00:00, 84.60step/s]INFO:autocal:InnerEvaluation(metric=RMSE):
                            values    n
name                                   
F3platform                0.297436  501
Helgoland                 1.224551  501
J61                       1.084463  501
K141                      0.286978  501
Westkapelle               0.507991  501
ZeebruggeWielingendok_wl  0.533548  501
L91                       0.302236  501
Processing: 100%|██████████| 500/500 [00:14<00:00, 35.09step/s]
INFO:autocal:InnerEvaluation(metric=RMSE):
                            values    n
name                                   
F3platform                0.309701  451
Helgoland                 1.264383  451
J61                       1.125848  451
K141                      0.278207  451
Westka

KeyboardInterrupt: 

##### For testing
Trial 10 should have an objective value of `0.5940855401333399 `